In [ ]:
import os
import re
import json
from datetime import datetime
import pandas as pd
import numpy as np

from typing import List, Dict, Tuple

In [ ]:
# File Paths - UPDATE THESE IF FILES ARE IN DIFFERENT LOCATIONS
AUG25_FILE = "/home/supriyo/Downloads/Biocon_nw/Aug'25_L2_DP_Plan_Circulation_V2.xlsx"
SNP_FILE = "/home/supriyo/Downloads/Biocon_nw/ParkourSC_SNP.xlsx"
BOM_FILE = "/home/supriyo/Downloads/Biocon_nw/20251006-DP Material Shortage - Working file.xlsx"
OUTPUT_FILE = "/mnt/user-data/outputs/Demand_Planning_Analysis.xlsx"

# Sheet Names
AUG25_SHEET = "L2Ph1_Detail"
SNP_SHEET = "DP RCCP"
BOM_SHEET = "DP Shortage"

# Products Configuration
PRODUCTS_AVAILABLE = [700001012, 700001123, 700000536, 700001318, 700001301]
PRODUCTS_NOT_AVAILABLE = [700004130]

print("✓ Configuration loaded")
print(f"  Input files: 3")
print(f"  Output file: {OUTPUT_FILE}")
print(f"  Products to analyze: {len(PRODUCTS_AVAILABLE)}")

# ----- Input files & sheet configuration -----
dp_file = "/home/supriyo/Downloads/Biocon_nw/Aug'25_L2_DP_Plan_Circulation_V2.xlsx"
dp_sheet = "L2Ph1_Detail"
dp_rows_start = 3
dp_rows_end = 78  # inclusive
market_sku_col = "E"
batch_size_col = "K"
product_id_col_hint = "Product_Id"  # we'll attempt to infer if not found

In [ ]:


# EA / Batches columns (Excel letter ranges)
ea_col_range = ("S", "AM")  # col S to AM (Oct-2025 .. Jun-2027) (21 months)
batch_col_range = ("AN", "BI")  # col AN to BI (often 21 months; BI may be an extra/blank col in some files)

# Output sheet for product demand
output_sheet_name = "Product_Demand"

# DP RCCP file
rccp_file = "/home/supriyo/Downloads/Biocon_nw/ParkourSC_SNP.xlsx"
rccp_sheet = "DP RCCP"
rccp_product_id_col = "C"  # Product ID in column C (row 3 onward)
rccp_key_figure_col = "H"  # "Planned Production" / "Outbound Production Demand"
rccp_months_range = ("J", "AD")  # Mar-25 .. Apr-27 (24M is mentioned; actual columns ~21)

# DP Shortage file for BOM mapping
shortage_file = "/home/supriyo/Downloads/Biocon_nw/20251006-DP Material Shortage - Working file.xlsx"
shortage_sheet = "DP Shortage"
shortage_filter_rows = (23, 542)  # only use this slice
shortage_product_id_col = "A"  # Product ID
shortage_component_col = "F"   # Component ID / Material

# Filled FG available in hierarchy
filled_fg_available = ['700001012','700001123','700000536','700001318','700001301']
filled_fg_not_available = ['700004130']  # explicitly not available

# Provided product mapping (SKUs to hierarchy nodes)
product_mapping = {}

mCB_skus = ['800004403', '800004402', '800008019', '800008020', '800008034', '800007997', '800007345', '800007516',
            '800002513', '800007608', '800007630', '800002984', '800004986', '800007310', '800007311', '800006648',
            '800007634', '800008073', '800006523', '800002297', '800002872', '800006741', '800007380']

sMCB_skus = ['800006506', '800006505', '800006527', '800006526', '800006525', '800007546', '800007583', '800007839',
             '800006524', '800006627', '800007872']

vial_skus = ['800004400', '800004401', '800006626', '800006740', '800007996']

aspart_dlp_skus = ['800008016', '800002958', '800002948', '800006528', '800002989', '800003528', '800006592', '800006691']

aspart_vial_skus = ['800008017', '800006529']

rhi_skus = ['800001300', '800001298', '800001299']

for sku in mCB_skus:
    product_mapping[sku] = {'assembly': '700003964', 'filling': '700001012', 'root': '700001470', 'family': 'Glargine_mCB_DLP'}

for sku in sMCB_skus:
    product_mapping[sku] = {'assembly': '700004129', 'filling': '700004130', 'root': '700004130', 'family': 'Glargine_sMCB_DLP_EU'}

for sku in vial_skus:
    product_mapping[sku] = {'assembly': '700001123', 'filling': '700001123', 'root': '700001123', 'family': 'Glargine_Vial'}

for sku in aspart_dlp_skus:
    product_mapping[sku] = {'assembly': '700002770', 'filling': '700001301', 'root': '700001301', 'family': 'Aspart_DLP'}

for sku in aspart_vial_skus:
    product_mapping[sku] = {'assembly': '700001318', 'filling': '700001318', 'root': '700001318', 'family': 'Aspart_Vial'}

for sku in rhi_skus:
    product_mapping[sku] = {'assembly': '700000536', 'filling': '700000536', 'root': '700000536', 'family': 'RHI'}

# ---------- Helper functions ----------

def excel_col_to_index(col_letter: str) -> int:
    """Convert Excel column letter (e.g., 'S') to 0-based pandas index."""
    col_letter = col_letter.upper()
    exp = 0
    col_index = 0
    for char in reversed(col_letter):
        col_index += (ord(char) - ord('A') + 1) * (26 ** exp)
        exp += 1
    return col_index - 1  # 0-based

MONTH_ABBR_TO_NUM = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

def normalize_month_header(name: str) -> str:
    """
    Convert e.g., 'OCT 2025' -> '10.2025'; 'Mar-25' -> '3.2025'
    Returns original string if it doesn't match expected patterns.
    """
    if pd.isna(name):
        return name
    s = str(name).strip()
    # Pattern 1: 'OCT 2025'
    m = re.match(r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z]*\s+(\d{4})$', s, flags=re.IGNORECASE)
    if m:
        mon = m.group(1)[:3].upper()
        yr = int(m.group(2))
        return f"{MONTH_ABBR_TO_NUM[mon]}.{yr}"

    # Pattern 2: 'Mar-25' or 'MAR-25'
    m = re.match(r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z]*[-_/](\d{2,4})$', s, flags=re.IGNORECASE)
    if m:
        mon = m.group(1)[:3].upper()
        y = m.group(2)
        yr = int(y) + 2000 if len(y) == 2 else int(y)
        return f"{MONTH_ABBR_TO_NUM[mon]}.{yr}"

    # Pattern 3: already in '10.2025' or '10/2025' etc.
    m = re.match(r'^(\d{1,2})[\./-](\d{2,4})$', s)
    if m:
        mon = int(m.group(1))
        y = m.group(2)
        yr = int(y) + 2000 if len(y) == 2 else int(y)
        return f"{mon}.{yr}"

    return s

def restrict_rows(df: pd.DataFrame, start_row: int, end_row: int) -> pd.DataFrame:
    """
    Restrict a dataframe to Excel-like inclusive 1-based rows.
    start_row/end_row are Excel row numbers. Pandas is 0-based; assume header is row 1.
    """
    # If original data includes header as first row, data starts at Excel row 2.
    # We will slice using iloc with start_row-1 to end_row-1 (0-based inclusive slice end handled via +1)
    return df.iloc[start_row-1:end_row]

def find_col_by_letter(df: pd.DataFrame, letter: str) -> str:
    """Return the df column name located at the given Excel column letter."""
    idx = excel_col_to_index(letter)
    if idx < 0 or idx >= len(df.columns):
        raise IndexError(f"Column letter {letter} resolves to index {idx}, out of bounds for dataframe with {len(df.columns)} columns.")
    return df.columns[idx]

def convert_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Convert all columns to numeric where possible."""
    return df.apply(pd.to_numeric, errors='coerce')

def melt_months(df: pd.DataFrame, id_vars: List[str], month_cols: List[str], value_name: str) -> pd.DataFrame:
    """Melt month columns to a long format with normalized Month."""
    tmp = df[id_vars + month_cols].copy()
    melted = tmp.melt(id_vars=id_vars, value_vars=month_cols, var_name="MonthRaw", value_name=value_name)
    melted["Month"] = melted["MonthRaw"].apply(normalize_month_header)
    melted.drop(columns=["MonthRaw"], inplace=True)
    # ensure numeric
    melted[value_name] = pd.to_numeric(melted[value_name], errors="coerce").fillna(0)
    return melted

# ---------- Processing DP file (EA/Batches from L2Ph1_Detail) ----------

def process_dp_file() -> Tuple[pd.DataFrame, Dict[str, str]]:
    if not os.path.exists(dp_file):
        print(f"DP file not found at: {dp_file}. Proceeding with empty template.")
        return pd.DataFrame(columns=["Market_SKU","Batch_Size","Product_Id","Family","Month","EA","Batches"]), {}

    df = pd.read_excel(dp_file, sheet_name=dp_sheet, header=0)
    # Restrict to specified Excel rows (3..78 inclusive), excluding header row
    df_slice = restrict_rows(df, dp_rows_start, dp_rows_end)

    # Derive column names by letters
    col_market_sku = find_col_by_letter(df, market_sku_col)
    col_batch_size = find_col_by_letter(df, batch_size_col)

    # Month columns ranges
    ea_start, ea_end = excel_col_to_index(ea_col_range[0]), excel_col_to_index(ea_col_range[1])
    batch_start, batch_end = excel_col_to_index(batch_col_range[0]), excel_col_to_index(batch_col_range[1])

    ea_month_cols = list(df.columns[ea_start:ea_end+1])
    batch_month_cols = list(df.columns[batch_start:batch_end+1])

    # Normalize month headers preemptively
    ea_month_norm = [normalize_month_header(c) for c in ea_month_cols]
    batch_month_norm = [normalize_month_header(c) for c in batch_month_cols]

    # Coerce numeric for month columns
    df_slice[ea_month_cols] = convert_numeric(df_slice[ea_month_cols])
    df_slice[batch_month_cols] = convert_numeric(df_slice[batch_month_cols])

    # Build ID vars and melt
    id_vars = [col_market_sku, col_batch_size]

    ea_long = melt_months(df_slice, id_vars=id_vars, month_cols=ea_month_cols, value_name="EA")
    batch_long = melt_months(df_slice, id_vars=id_vars, month_cols=batch_month_cols, value_name="Batches")

    # Merge EA and Batches on identifiers + Month
    merged = pd.merge(
        ea_long[[col_market_sku, col_batch_size, "Month", "EA"]],
        batch_long[[col_market_sku, col_batch_size, "Month", "Batches"]],
        on=[col_market_sku, col_batch_size, "Month"],
        how="outer"
    ).fillna(0)

    # Rename id columns
    merged.rename(columns={col_market_sku: "Market_SKU", col_batch_size: "Batch_Size"}, inplace=True)

    # Map Market_SKU to Product_Id via provided mapping (use 'filling' as Product_Id by default when available)
    def map_product_id(sku: str) -> Tuple[str, str]:
        sk = str(sku) if not pd.isna(sku) else ""
        info = product_mapping.get(sk, {})
        product = info.get("filling") or info.get("assembly") or info.get("root")
        fam = info.get("family")
        return product, fam

    mapped = merged.copy()
    mapped[["Product_Id","Family"]] = mapped["Market_SKU"].apply(lambda x: pd.Series(map_product_id(x)))
    # Filter rows where Product_Id is present
    mapped = mapped[~mapped["Product_Id"].isna()]

    # Ensure Month is within Oct-2025 .. Jun-2027 inclusive
    def month_to_tuple(mstr: str):
        try:
            m, y = mstr.split(".")
            return int(y), int(m)
        except Exception:
            return (0, 0)

    start_key = (2025, 10)
    end_key = (2027, 6)
    mapped = mapped[mapped["Month"].apply(lambda v: start_key <= month_to_tuple(v) <= end_key)]

    # Sort
    mapped.sort_values(by=["Market_SKU","Month"], inplace=True)
    return mapped, {"ea_cols": ea_month_cols, "batch_cols": batch_month_cols}

# ---------- Processing DP RCCP (Planned Production / Outbound Production Demand) ----------

def process_rccp_file() -> Dict[str, pd.DataFrame]:
    results = {}
    if not os.path.exists(rccp_file):
        print(f"RCCP file not found at: {rccp_file}. Proceeding with empty templates.")
        results["PlannedProduction"] = pd.DataFrame(columns=["Product_ID","Key_Figure","Base_UOM","Scenario","Month","Qty"])
        results["OutboundProductionDemand"] = pd.DataFrame(columns=["Product_ID","Key_Figure","Base_UOM","Scenario","Month","Qty"])
        return results

    # Read with headers
    rccp_df = pd.read_excel(rccp_file, sheet_name=rccp_sheet, header=0)

    # Try to infer named columns (since provided are letters)
    def colname_by_letter(df, letter):
        try:
            return find_col_by_letter(df, letter)
        except Exception:
            return None

    col_prod = colname_by_letter(rccp_df, rccp_product_id_col) or "Product ID"
    col_keyf = colname_by_letter(rccp_df, rccp_key_figure_col) or "Key Figure"

    # Month columns by letters
    m_start_idx = excel_col_to_index(rccp_months_range[0])
    m_end_idx = excel_col_to_index(rccp_months_range[1])
    rccp_month_cols = list(rccp_df.columns[m_start_idx:m_end_idx+1])

    # Identify optional helpful columns if present
    col_base_uom = "Base UOM" if "Base UOM" in rccp_df.columns else None
    col_scenario = "Scenario" if "Scenario" in rccp_df.columns else None

    # Coerce month columns numeric
    rccp_df[rccp_month_cols] = convert_numeric(rccp_df[rccp_month_cols])

    id_vars = [c for c in [col_prod, col_keyf, col_base_uom, col_scenario] if c]

    rccp_long = melt_months(rccp_df, id_vars=id_vars, month_cols=rccp_month_cols, value_name="Qty")

    # Split by Key Figure
    if col_keyf in rccp_long.columns:
        pp = rccp_long[rccp_long[col_keyf].str.lower().eq("planned production", na=False)].copy()
        opd = rccp_long[rccp_long[col_keyf].str.lower().eq("outbound production demand", na=False)].copy()
    else:
        pp = rccp_long.iloc[0:0].copy()
        opd = rccp_long.iloc[0:0].copy()

    # Normalize column names
    def tidy(df):
        out = df.rename(columns={col_prod: "Product_ID", col_keyf: "Key_Figure"})
        if col_base_uom and col_base_uom in out.columns:
            out.rename(columns={col_base_uom: "Base_UOM"}, inplace=True)
        else:
            out["Base_UOM"] = np.nan
        if col_scenario and col_scenario in out.columns:
            out.rename(columns={col_scenario: "Scenario"}, inplace=True)
        else:
            out["Scenario"] = np.nan
        # Keep only the window Mar-2025 .. Apr-2027 if present
        def month_tuple(mstr):
            try:
                m, y = mstr.split(".")
                return int(y), int(m)
            except Exception:
                return (0, 0)
        start_key = (2025, 3)
        end_key = (2027, 4)
        out = out[out["Month"].apply(lambda v: start_key <= month_tuple(v) <= end_key)]
        return out[["Product_ID","Key_Figure","Base_UOM","Scenario","Month","Qty"]]

    results["PlannedProduction"] = tidy(pp)
    results["OutboundProductionDemand"] = tidy(opd)
    return results

# ---------- BOM mapping using DP Shortage ----------

def bom_demand_from_shortage(opd_long: pd.DataFrame) -> pd.DataFrame:
    """
    Using DP Shortage to map from a Finished Good Product_ID to its component materials (Component IDs),
    then aggregate Outbound Production Demand from RCCP for those components, per month.
    """
    if opd_long is None or opd_long.empty:
        return pd.DataFrame(columns=["FG_Product_ID","Component_ID","Month","Outbound_Prod_Demand"])

    if not os.path.exists(shortage_file):
        print(f"Shortage file not found at: {shortage_file}. Proceeding with empty template for BOM demand.")
        return pd.DataFrame(columns=["FG_Product_ID","Component_ID","Month","Outbound_Prod_Demand"])

    # Load Shortage sheet; slice rows 23..542 (Excel numbering; header at 1), so pandas rows 22..541 (0-based)
    sh_df = pd.read_excel(shortage_file, sheet_name=shortage_sheet, header=0)
    sh_df = sh_df.iloc[shortage_filter_rows[0]-1:shortage_filter_rows[1]]

    # Infer column names by letters (A=Product_ID, F=Component_ID)
    try:
        col_prodA = find_col_by_letter(sh_df, shortage_product_id_col)
    except Exception:
        # fallback find by name
        col_prodA = "Product_ID" if "Product_ID" in sh_df.columns else (sh_df.columns[0] if len(sh_df.columns)>0 else None)

    try:
        col_compF = find_col_by_letter(sh_df, shortage_component_col)
    except Exception:
        # fallback by common names
        col_compF = "Component_ID" if "Component_ID" in sh_df.columns else (sh_df.columns[5] if len(sh_df.columns)>5 else None)

    if not col_prodA or not col_compF:
        # Cannot proceed properly
        return pd.DataFrame(columns=["FG_Product_ID","Component_ID","Month","Outbound_Prod_Demand"])

    # Remove blanks / zeros
    sh_df = sh_df[(sh_df[col_prodA].notna()) & (sh_df[col_prodA] != 0) & (sh_df[col_prodA] != "")]

    # Focus only on the FG list provided (available ones)
    sh_df = sh_df[sh_df[col_prodA].astype(str).isin(filled_fg_available)]

    # Build mapping FG -> set(Components)
    comp_map = (
        sh_df[[col_prodA, col_compF]]
        .dropna()
        .astype(str)
        .groupby(col_prodA)[col_compF]
        .agg(lambda s: sorted(set(s)))
        .to_dict()
    )

    # For each FG, sum Outbound Production Demand for its components per month
    rows = []
    for fg, components in comp_map.items():
        comp_rows = opd_long[opd_long["Product_ID"].astype(str).isin(components)]
        if comp_rows.empty:
            continue
        agg = comp_rows.groupby("Month", as_index=False)["Qty"].sum()
        for _, r in agg.iterrows():
            rows.append({
                "FG_Product_ID": fg,
                "Component_ID": ";".join(components),
                "Month": r["Month"],
                "Outbound_Prod_Demand": r["Qty"]
            })

    result = pd.DataFrame(rows)
    # Keep only the provided FG list
    if not result.empty:
        result = result[result["FG_Product_ID"].isin(filled_fg_available)]
        # Sort
        def key(mstr):
            try:
                m, y = mstr.split(".")
                return (int(y), int(m))
            except Exception:
                return (0, 0)
        result.sort_values(by=["FG_Product_ID","Month"], key=lambda s: s.map(key), inplace=True)
    return result

# ---------- Run end-to-end and write outputs ----------

product_demand_long, dp_meta = process_dp_file()
rccp_results = process_rccp_file()
planned_prod = rccp_results.get("PlannedProduction", pd.DataFrame())
opd = rccp_results.get("OutboundProductionDemand", pd.DataFrame())
bom_from_shortage = bom_demand_from_shortage(opd)

# Prepare a compact "example" view for the requested header format:
# Month | EA | Batches (for any one SKU as example if present)
def compact_view(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=["Month","EA","Batches"])
    # Take first SKU present
    sku = df["Market_SKU"].iloc[0]
    sub = df[df["Market_SKU"] == sku][["Month","EA","Batches"]].copy()
    sub = sub.groupby("Month", as_index=False).sum()
    return sub

compact = compact_view(product_demand_long)

# Write to an output Excel file with multiple sheets
out_path = "/mnt/data/DP_RCCP_mapped_output.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    product_demand_long.to_excel(writer, sheet_name=output_sheet_name, index=False)
    planned_prod.to_excel(writer, sheet_name="RCCP_PlannedProduction", index=False)
    opd.to_excel(writer, sheet_name="RCCP_OutboundProdDemand", index=False)
    bom_from_shortage.to_excel(writer, sheet_name="BOM_OPD_Demand", index=False)

# Display previews (if non-empty)
try:
    from ace_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

if display_dataframe_to_user:
    if not product_demand_long.empty:
        display_dataframe_to_user("Product_Demand (Oct-2025 to Jun-2027)", product_demand_long.head(50))
    if not planned_prod.empty:
        display_dataframe_to_user("DP RCCP - Planned Production (Mar-2025 to Apr-2027)", planned_prod.head(50))
    if not opd.empty:
        display_dataframe_to_user("DP RCCP - Outbound Production Demand (Mar-2025 to Apr-2027)", opd.head(50))
    if not bom_from_shortage.empty:
        display_dataframe_to_user("BOM Demand from Shortage (OPD aggregated per FG)", bom_from_shortage.head(50))
    if not compact.empty:
        display_dataframe_to_user("Compact Month | EA | Batches (first SKU example)", compact)

out_path
